In [1]:
import requests
import math 

from io import BytesIO

import vnquant.data as dt
import pandas as pd

In [2]:
def get_data(ticker):
    """
    Hàm dùng để lấy data từ API của cafeF thông qua vnquant
    ticker là mã chứng khoán
    Data trả về là dữ liệu lịch sử về giá của cổ phiếu
    """
    loader = dt.DataLoader(
        symbols=ticker, 
        start="2022-01-01", 
        end="2022-06-30", 
        minimal=False, 
        data_source="cafe"
    )

    data = loader.download()

    return data

In [3]:
def get_general_data(url):
    """
    Hàm dùng để lấy data từ file google sheet
    url là link của sheet
    """
    r = requests.get(url)
    data = r.content
    return pd.read_csv(BytesIO(data), index_col=0, thousands=',').reset_index()

In [4]:
general_df = get_general_data(
    "https://docs.google.com/spreadsheets/d/e/"\
    "2PACX-1vQmN9UGZ1PZpBkNtsXRGU4ThR0cMrNkLamoNO20hqbcf1DWPgQWE6zOl2htjdu34xawP25dheCyXMug/"\
    "pub?gid=12001416&single=true&output=csv"
)

In [5]:
general_df

,industry,company,ticker,market_cap
0,Dịch vụ,Công ty Cổ phần Nước – Môi trường Bình Dương,BWE,9858.21
1,Dịch vụ,Công ty Cổ phần In sách giáo khoa Hòa Phát,HTP,3314.16
2,Dịch vụ,Công ty Cổ phần Đầu tư Apax Holdings,IBC,1650.54
3,Dịch vụ,Tổng Công ty Cổ phần Dịch vụ Tổng hợp Dầu khí,PET,3273.11
4,Dịch vụ,Công ty Cổ phần Vàng bạc Đá quý Phú Nhuận,PNJ,31071.39
...,...,...,...,...
118,Bất động sản - Xây dựng,Công ty Cổ phần Thaiholdings,THD,19845.00
119,Bất động sản - Xây dựng,Tổng Công ty Viglacera - CTCP,VGC,23717.72
120,Bất động sản - Xây dựng,Công ty cổ phần Vinhomes,VHM,267358.16
121,Bất động sản - Xây dựng,Tập đoàn Vingroup - Công ty Cổ phần,VIC,277524.85


In [6]:
# Lấy ra một list các mã chứng khoán trong sheet, để từ đó call API cafeF để lấy dữ liệu lịch sử
list_tickers = list(general_df["ticker"])

In [7]:
len(list_tickers)

123

In [8]:
df = get_data(ticker=list_tickers)

2022-07-23 16:28:03,633 : INFO : NumExpr defaulting to 4 threads.
/Users/phamcuong/opt/miniconda3/lib/python3.8/site-packages/vnquant-0.0.3-py3.8.egg/vnquant/data/dataloader.py:256: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  stock_data['change_perc1'], stock_data['change_perc2'] = stock_data['change_perc'].apply(utils.split_change_col).str
2022-07-23 16:28:05,409 : INFO : data BWE from 2022-01-01 to 2022-06-30 have already cloned!
/Users/phamcuong/opt/miniconda3/lib/python3.8/site-packages/vnquant-0.0.3-py3.8.egg/vnquant/data/dataloader.py:256: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  stock_data['change_perc1'], stock_data['change_perc2'] = stock_data['change_perc'].apply(utils.split_change_col).str
2022-07-23 16:28:07,143 : INFO : data HTP from 2022-01-01 to 2022-06-30 have already cloned!
/Users/phamcuong/opt/miniconda3/lib/python3.8/site-packages/vnquant-0.0.3-py3.8.egg/vnquant/data/dat

In [9]:
stacked_df = df[["close"]].stack().reset_index()

In [10]:
# Dữ liệu giá đóng cửa của các cổ phiếu nhưng được transform thành dạng dọc (thay vì dạng ngang như ban đầu)
stacked_df

Attributes,date,Symbols,close
0,2022-01-04,AAA,20.70
1,2022-01-04,ACB,34.40
2,2022-01-04,ANV,34.10
3,2022-01-04,APH,37.65
4,2022-01-04,AST,53.20
...,...,...,...
14635,2022-06-30,VPB,29.00
14636,2022-06-30,VPG,29.00
14637,2022-06-30,VRE,28.55
14638,2022-06-30,VSC,41.65


In [11]:
# JOIN stacked_df với df ban đầu để lấy data về market size cho từng cổ phiếu
merged_df = pd.merge(stacked_df, general_df, how="left", left_on='Symbols', right_on='ticker')

In [12]:
# Lấy ngày, tên cổ phiếu, ngành, market cap, và giá đóng cửa
merged_df = merged_df[["date", "ticker", "industry", "market_cap", "close"]]

In [13]:
# Pivot merged_df để tính toán phân trăm thay đổi giá cổ phiếu giữa ngày thứ X và ngày đầu tiên
pivot_df = merged_df.pivot(index=["ticker", "industry", "market_cap"], columns="date", values="close").reset_index()

In [14]:
pivot_df

date,ticker,industry,market_cap,2022-01-04 00:00:00,2022-01-05 00:00:00,2022-01-06 00:00:00,2022-01-07 00:00:00,2022-01-10 00:00:00,2022-01-11 00:00:00,2022-01-12 00:00:00,...,2022-06-17 00:00:00,2022-06-20 00:00:00,2022-06-21 00:00:00,2022-06-22 00:00:00,2022-06-23 00:00:00,2022-06-24 00:00:00,2022-06-27 00:00:00,2022-06-28 00:00:00,2022-06-29 00:00:00,2022-06-30 00:00:00
0,AAA,Vật liệu,3737.67,20.70,20.95,22.40,22.80,21.25,20.00,19.60,...,11.00,10.30,11.0,11.35,11.70,11.80,11.90,12.00,11.85,11.30
1,ACB,Tài chính,81227.31,34.40,33.70,33.75,33.30,33.00,32.75,33.10,...,23.10,22.90,23.0,23.75,23.75,23.80,23.95,24.50,24.20,24.00
2,ANV,Nông nghiệp,6817.54,34.10,35.85,35.40,35.30,33.45,32.00,31.15,...,63.70,61.70,57.4,53.40,57.10,57.00,57.50,53.50,56.00,53.00
3,APH,Vật liệu,2975.39,37.65,37.15,37.25,37.00,35.95,34.50,34.70,...,11.65,10.85,10.7,11.40,11.65,12.20,12.45,12.80,12.80,12.00
4,AST,Công nghiệp - Hàng không,2605.50,53.20,53.50,53.50,53.00,53.30,52.70,52.50,...,57.90,57.70,56.0,56.00,56.70,56.10,58.50,58.70,58.30,57.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,VPB,Tài chính,128461.18,36.05,35.50,35.15,34.60,34.40,34.30,35.40,...,27.90,27.10,27.2,28.40,28.45,28.15,28.90,30.00,30.00,29.00
119,VPG,Vật liệu,2341.79,57.10,56.10,55.50,55.00,53.00,53.20,53.00,...,31.75,29.55,27.5,28.80,29.75,30.15,30.35,30.25,29.70,29.00
120,VRE,Bất động sản - Xây dựng,65101.92,31.15,33.30,35.60,34.75,35.00,34.20,36.00,...,29.35,28.70,28.0,28.50,28.05,27.55,28.75,28.85,28.45,28.55
121,VSC,Công nghiệp - Hàng không,5032.71,43.50,43.00,42.70,43.30,42.70,41.70,40.40,...,50.20,47.50,45.6,42.45,45.30,44.30,44.90,44.00,42.20,41.65


In [15]:
# Tạo một df mới lấy từ 3 cột của pivot_df, dùng df này để thêm % thay đổi của các ngày
final_result = pivot_df[["ticker", "industry", "market_cap"]]

In [16]:
arr = []
# Cleaning tên cột của pivot_df, vì tên cột dạng datetime đang thừa phần 00:00:00
for column in pivot_df:
    arr.append(str(column).split(" ")[0])

In [17]:
arr

['ticker',
 'industry',
 'market_cap',
 '2022-01-04',
 '2022-01-05',
 '2022-01-06',
 '2022-01-07',
 '2022-01-10',
 '2022-01-11',
 '2022-01-12',
 '2022-01-13',
 '2022-01-14',
 '2022-01-17',
 '2022-01-18',
 '2022-01-19',
 '2022-01-20',
 '2022-01-21',
 '2022-01-24',
 '2022-01-25',
 '2022-01-26',
 '2022-01-27',
 '2022-01-28',
 '2022-02-07',
 '2022-02-08',
 '2022-02-09',
 '2022-02-10',
 '2022-02-11',
 '2022-02-14',
 '2022-02-15',
 '2022-02-16',
 '2022-02-17',
 '2022-02-18',
 '2022-02-21',
 '2022-02-22',
 '2022-02-23',
 '2022-02-24',
 '2022-02-25',
 '2022-02-28',
 '2022-03-01',
 '2022-03-02',
 '2022-03-03',
 '2022-03-04',
 '2022-03-07',
 '2022-03-08',
 '2022-03-09',
 '2022-03-10',
 '2022-03-11',
 '2022-03-14',
 '2022-03-15',
 '2022-03-16',
 '2022-03-17',
 '2022-03-18',
 '2022-03-21',
 '2022-03-22',
 '2022-03-23',
 '2022-03-24',
 '2022-03-25',
 '2022-03-28',
 '2022-03-29',
 '2022-03-30',
 '2022-03-31',
 '2022-04-01',
 '2022-04-04',
 '2022-04-05',
 '2022-04-06',
 '2022-04-07',
 '2022-04-08',
 

In [18]:
# Đổi tên cột của pivot_df với những cột vừa thay 
pivot_df.columns = arr

In [19]:
# Ngày đầu tiên là mốc thời gian để so sánh giá, vì thế ngày này, mức độ thay đổi mặc định là 0
final_result[arr[3]] = 0

<ipython-input-19-36f657fb020d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result[arr[3]] = 0


In [20]:
# Tính toán sự thay đổi của các ngày so với ngày đầu tiên
for i in range(3, len(arr)):
    final_result[arr[i]] = round((pivot_df[arr[i]] - pivot_df[arr[3]]) / pivot_df[arr[3]] * 100, 2)

<ipython-input-20-b76f10c022ba>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_result[arr[i]] = round((pivot_df[arr[i]] - pivot_df[arr[3]]) / pivot_df[arr[3]] * 100, 2)


In [21]:
final_result

date,ticker,industry,market_cap,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-10,2022-01-11,2022-01-12,...,2022-06-17,2022-06-20,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-27,2022-06-28,2022-06-29,2022-06-30
0,AAA,Vật liệu,3737.67,0.0,1.21,8.21,10.14,2.66,-3.38,-5.31,...,-46.86,-50.24,-46.86,-45.17,-43.48,-43.00,-42.51,-42.03,-42.75,-45.41
1,ACB,Tài chính,81227.31,0.0,-2.03,-1.89,-3.20,-4.07,-4.80,-3.78,...,-32.85,-33.43,-33.14,-30.96,-30.96,-30.81,-30.38,-28.78,-29.65,-30.23
2,ANV,Nông nghiệp,6817.54,0.0,5.13,3.81,3.52,-1.91,-6.16,-8.65,...,86.80,80.94,68.33,56.60,67.45,67.16,68.62,56.89,64.22,55.43
3,APH,Vật liệu,2975.39,0.0,-1.33,-1.06,-1.73,-4.52,-8.37,-7.84,...,-69.06,-71.18,-71.58,-69.72,-69.06,-67.60,-66.93,-66.00,-66.00,-68.13
4,AST,Công nghiệp - Hàng không,2605.50,0.0,0.56,0.56,-0.38,0.19,-0.94,-1.32,...,8.83,8.46,5.26,5.26,6.58,5.45,9.96,10.34,9.59,8.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,VPB,Tài chính,128461.18,0.0,-1.53,-2.50,-4.02,-4.58,-4.85,-1.80,...,-22.61,-24.83,-24.55,-21.22,-21.08,-21.91,-19.83,-16.78,-16.78,-19.56
119,VPG,Vật liệu,2341.79,0.0,-1.75,-2.80,-3.68,-7.18,-6.83,-7.18,...,-44.40,-48.25,-51.84,-49.56,-47.90,-47.20,-46.85,-47.02,-47.99,-49.21
120,VRE,Bất động sản - Xây dựng,65101.92,0.0,6.90,14.29,11.56,12.36,9.79,15.57,...,-5.78,-7.87,-10.11,-8.51,-9.95,-11.56,-7.70,-7.38,-8.67,-8.35
121,VSC,Công nghiệp - Hàng không,5032.71,0.0,-1.15,-1.84,-0.46,-1.84,-4.14,-7.13,...,15.40,9.20,4.83,-2.41,4.14,1.84,3.22,1.15,-2.99,-4.25


In [22]:
# Fill các giá trị trống NAN bằng 0
final_result = final_result.fillna(0)

In [23]:
final_result

date,ticker,industry,market_cap,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-10,2022-01-11,2022-01-12,...,2022-06-17,2022-06-20,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-27,2022-06-28,2022-06-29,2022-06-30
0,AAA,Vật liệu,3737.67,0.0,1.21,8.21,10.14,2.66,-3.38,-5.31,...,-46.86,-50.24,-46.86,-45.17,-43.48,-43.00,-42.51,-42.03,-42.75,-45.41
1,ACB,Tài chính,81227.31,0.0,-2.03,-1.89,-3.20,-4.07,-4.80,-3.78,...,-32.85,-33.43,-33.14,-30.96,-30.96,-30.81,-30.38,-28.78,-29.65,-30.23
2,ANV,Nông nghiệp,6817.54,0.0,5.13,3.81,3.52,-1.91,-6.16,-8.65,...,86.80,80.94,68.33,56.60,67.45,67.16,68.62,56.89,64.22,55.43
3,APH,Vật liệu,2975.39,0.0,-1.33,-1.06,-1.73,-4.52,-8.37,-7.84,...,-69.06,-71.18,-71.58,-69.72,-69.06,-67.60,-66.93,-66.00,-66.00,-68.13
4,AST,Công nghiệp - Hàng không,2605.50,0.0,0.56,0.56,-0.38,0.19,-0.94,-1.32,...,8.83,8.46,5.26,5.26,6.58,5.45,9.96,10.34,9.59,8.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,VPB,Tài chính,128461.18,0.0,-1.53,-2.50,-4.02,-4.58,-4.85,-1.80,...,-22.61,-24.83,-24.55,-21.22,-21.08,-21.91,-19.83,-16.78,-16.78,-19.56
119,VPG,Vật liệu,2341.79,0.0,-1.75,-2.80,-3.68,-7.18,-6.83,-7.18,...,-44.40,-48.25,-51.84,-49.56,-47.90,-47.20,-46.85,-47.02,-47.99,-49.21
120,VRE,Bất động sản - Xây dựng,65101.92,0.0,6.90,14.29,11.56,12.36,9.79,15.57,...,-5.78,-7.87,-10.11,-8.51,-9.95,-11.56,-7.70,-7.38,-8.67,-8.35
121,VSC,Công nghiệp - Hàng không,5032.71,0.0,-1.15,-1.84,-0.46,-1.84,-4.14,-7.13,...,15.40,9.20,4.83,-2.41,4.14,1.84,3.22,1.15,-2.99,-4.25


In [24]:
# Export final data ra file csv
final_result.to_csv("data.csv")